# CSC_52081_EP Project

Advanced Machine Learning and Autonomous Agents Project

## Introduction

Reinforcement Learning (RL) has emerged as a robust framework for training autonomous agents to learn optimal behaviors through environmental interactions. This study utilizes the [`CarRacing-v3`](https://gymnasium.farama.org/environments/box2d/car_racing/) environment from Gymnasium, which presents a challenging control task in a racing scenario.

### Environment

The environment features a high-dimensional observation space, represented by a $96 \times 96$ RGB image capturing the car and track, necessitating the use of deep convolutional neural networks (CNNs) for effective feature extraction.

#### Action Space

The action space in CarRacing-v3 supports both continuous and discrete control modes.

In **continuous mode**, the agent outputs three real-valued commands:

- steering (ranging from $-1$ to $+1$)
- gas
- braking

In **discrete mode**, the action space is simplified to five actions:

- do nothing
- steer left
- steer right
- gas
- brake

This dual action representation enables a comprehensive evaluation of various RL algorithms under different control settings.

#### Reward

The reward structure combines a penalty of $-0.1$ per frame and a reward of $+\frac{1000}{N}$ for each new track tile visited, where $N$ is the total number of tiles. This incentivizes the agent to balance exploration (visiting tiles) with efficiency (minimizing frame usage). For example, completing the race after visiting all $N$ tiles in 732 frames yields a reward of $1000 - 0.1 \times 732 = 926.8$ points.

### Objective

The primary objective of this project is to compare RL policies across discrete and continuous action modalities. This comparative analysis aims to understand the strengths and limitations of each method in handling complex decision spaces.

The high-dimensional visual inputs in `CarRacing-v3` require effective feature extraction, addressed through a tailored CNN architecture. Transitioning between discrete and continuous action representations also demands careful algorithmic design and parameter tuning to ensure stable learning and convergence. While prior studies have often focused on either discrete or continuous action spaces separately, this work adopts a comparative approach, evaluating different agents within the same environment to assess performance under similar conditions.

At this stage, the work outlines the methodology and anticipated challenges, focusing on designing the CNN-based feature extractor, implementing RL algorithms, and establishing a framework for performance comparison. Preliminary findings are yet to be finalized, but the study is expected to provide insights into applying RL in high-dimensional, real-time control tasks. Limitations include the preliminary nature of experiments and the need for further tuning and validation. Future work will involve extensive empirical evaluations, exploring additional policy gradient methods, and refining the network architecture to better handle the complexities of `CarRacing-v3`.

#### Continuos

Approaches:

- **Cross-Entropy Method** (CEM)
- **Self-Adaptive Evolution Strategy** (SA-ES)
- **Proximal Policy Optimization** (PPO)
- **Soft Actor-Critic** (SAC)

#### Discrete

Approaches:

- **Deep Q-Network** (DQN)
- **SARSA**

### GitHub

The project's code is available on [GitHub](https://github.com/tr0fin0/ensta_CSC_52081_EP_project), offering a reproducible framework for future investigations and extensions.

## Installation

### Environment

#### WSL, Linux or MacOS

A `Python Virtual Environment` will be used for this project by run the following on a terminal on the project folder:

```bash
$ sudo apt install python3.10-venv
$ python3 -m venv env
$ source env/bin/activate
(env) $ python3 -m pip install --upgrade pip
(env) $ python3 -m pip install -r requirements.txt
```

### Imports

In [12]:
from collections import deque
from ipywidgets import interact
from IPython.display import Video
from pathlib import Path
from tqdm.notebook import tqdm
from typing import cast, List, Tuple, Deque, Optional, Callable


import gymnasium as gym
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

In [14]:
sns.set_context("talk")

In [15]:
def video_selector(file_path: List[Path]) -> Video:
    return Video(file_path, embed=True, html_attributes="controls autoplay loop")

### Setup

#### Directories

In [16]:
DIRECTORY_OUTPUT = "output"
DIRECTORY_MODELS = Path(f"{DIRECTORY_OUTPUT}/models/")
DIRECTORY_FIGURES = Path(f"{DIRECTORY_OUTPUT}/images/")

if not DIRECTORY_FIGURES.exists():
    DIRECTORY_FIGURES.mkdir(parents=True)

if not DIRECTORY_MODELS.exists():
    DIRECTORY_MODELS.mkdir(parents=True)

VIDEO_DEMO = "CSC_52081_EP_demonstration"
(DIRECTORY_FIGURES / f"{VIDEO_DEMO}.mp4").unlink(missing_ok=True)

VIDEO_DQN = "CSC_52081_EP_dqn"
(DIRECTORY_FIGURES / f"{VIDEO_DQN}.mp4").unlink(missing_ok=True)

## Demonstration

In [17]:
env = gym.make(
    "CarRacing-v3",
    render_mode="rgb_array",
    lap_complete_percent=0.95,
    domain_randomize=False,
    continuous=False
)
env = gym.wrappers.RecordVideo(env, video_folder=str(DIRECTORY_FIGURES), name_prefix=VIDEO_DEMO)


done = False
observation, info = env.reset()

while not done:
    action = env.action_space.sample()

    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

env.close()


Video(
    DIRECTORY_FIGURES / f"{VIDEO_DEMO}-episode-0.mp4",
    embed=True,
    html_attributes="controls autoplay loop",
)

## Description

only demonstration is right. from below here is only experimental.

### Global Definitions

#### Constants

In [ ]:
STATE_SHAPE = (96, 96, 3)
CONTINUOUS_ACTIONS = 3
DISCRETE_ACTIONS = 5
EPISODES = 50

#### Functions

In [19]:
def plot_learning_curve(rewards):
    plt.plot(rewards)
    plt.xlabel("Episodes")
    plt.ylabel("Cumulative Reward")
    plt.title("Learning Curve")
    plt.show()

def plot_action_heatmap(action_counts):
    plt.imshow(action_counts, cmap="hot", interpolation="nearest")
    plt.colorbar()
    plt.title("Action Heatmap")
    plt.show()

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DQN(nn.Module):
    def __init__(self, DISCRETE_ACTIONS):
        super(DQN, self).__init__()                             # (1, 3, 96, 96),   input layer
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, DISCRETE_ACTIONS)             # Output Q-values for each action

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)   # Flatten
        x = F.relu(self.fc1(x))

        return self.fc2(x)          # Q-values for each action

In [ ]:
import random
from collections import deque
import numpy as np

class DQNAgent:
    def __init__(self, DISCRETE_ACTIONS, state_space, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995, lr=0.001, batch_size=64):
        self.DISCRETE_ACTIONS = DISCRETE_ACTIONS
        self.state_space = state_space
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.memory = deque(maxlen=100000)  # Experience replay buffer
        self.model = DQN(DISCRETE_ACTIONS)
        self.target_model = DQN(DISCRETE_ACTIONS)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.update_target_model()

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.DISCRETE_ACTIONS)  # Random action

        state = torch.FloatTensor(state).unsqueeze(0)  # Add batch dimension
        act_values = self.model(state)

        return torch.argmax(act_values).item()  # Greedy action

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        minibatch = random.sample(self.memory, self.batch_size)
        states = torch.FloatTensor(np.array([t[0] for t in minibatch]))
        actions = torch.LongTensor(np.array([t[1] for t in minibatch]))
        rewards = torch.FloatTensor(np.array([t[2] for t in minibatch]))
        next_states = torch.FloatTensor(np.array([t[3] for t in minibatch]))
        dones = torch.FloatTensor(np.array([t[4] for t in minibatch]))

        current_q = self.model(states).gather(1, actions.unsqueeze(1))
        next_q = self.target_model(next_states).detach().max(1)[0]
        target_q = rewards + (1 - dones) * self.gamma * next_q

        loss = F.mse_loss(current_q.squeeze(), target_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save(self, path):
        torch.save(self.model.state_dict(), path)

    def load(self, path):
        self.model.load_state_dict(torch.load(path))

: 

In [ ]:
import gymnasium as gym

env = gym.make(
    "CarRacing-v3",
    render_mode="rgb_array",
    lap_complete_percent=0.95,
    domain_randomize=False,
    continuous=False
)
state_space = env.observation_space.shape
agent = DQNAgent(DISCRETE_ACTIONS, state_space)

for episode in range(EPISODES):
    total_reward = 0

    done = False
    state, info = env.reset()
    state = np.transpose(state, (2, 0, 1))  # Reshape to (C, H, W)

    while not done:
        # print(type(state), state)

        action = agent.action(state)
        # print(type(action), action)
        next_state, reward, terminated, truncated, info = env.step(action)
        # next_state, reward, done, _ = env.step(np.int64(action))
        next_state = np.transpose(next_state, (2, 0, 1))
        done = terminated or truncated

        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if done:
            print(f"Episode: {episode + 1}, Reward: {total_reward}, Epsilon: {agent.epsilon}")
            break

        agent.replay()

    if episode % 10 == 0:
        agent.update_target_model()

agent.save("dqn_car_racing_discrete.pth")

Episode: 1, Reward: -66.44295302013488, Epsilon: 0.00998645168764533
Episode: 2, Reward: -23.588039867110197, Epsilon: 0.00998645168764533
Episode: 3, Reward: -13.978494623656593, Epsilon: 0.00998645168764533
Episode: 4, Reward: -76.11940298507452, Epsilon: 0.00998645168764533
Episode: 5, Reward: -36.02693602693642, Epsilon: 0.00998645168764533
Episode: 6, Reward: -32.53968253968326, Epsilon: 0.00998645168764533
Episode: 7, Reward: -45.73643410852785, Epsilon: 0.00998645168764533
Episode: 8, Reward: -65.98639455782366, Epsilon: 0.00998645168764533
Episode: 9, Reward: -80.98859315589316, Epsilon: 0.00998645168764533
Episode: 10, Reward: -54.24836601307233, Epsilon: 0.00998645168764533
Episode: 11, Reward: -55.1724137931044, Epsilon: 0.00998645168764533


In [ ]:
agent.load("dqn_car_racing_discrete.pth")
env = gym.wrappers.RecordVideo(env, video_folder=str(DIRECTORY_FIGURES), name_prefix=VIDEO_DQN)

total_reward = 0
done = False
state, info = env.reset()
state = np.transpose(state, (2, 0, 1))

while not done:
    action = agent.action(state)

    next_state, reward, terminated, truncated, info = env.step(action)
    next_state = np.transpose(next_state, (2, 0, 1))
    done = terminated or truncated

    state = next_state
    total_reward += reward

    env.render()
env.close()

print(f"Test Reward: {total_reward}")


Video(
    DIRECTORY_FIGURES / f"{VIDEO_DQN}-episode-0.mp4",
    embed=True,
    html_attributes="controls autoplay loop",
)


/home/trofino/git_repositories/classes_ensta/3A/CSC_52081_EP/CSC_52081_EP_project/src/env/lib/python3.10/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /home/trofino/git_repositories/classes_ensta/3A/CSC_52081_EP/CSC_52081_EP_project/src/output/images folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Test Reward: -59.18367346938824
